## Домашняя работа 6
фев'24
<hr>

### Часть 1. EDA

<br>Скачайте данные с Kaggle по ценам на жильё в Airbnb в Нью-Йорке:
<br>Пройдите по основным шагам работы с данными:
<ul><li>выкиньте ненужные признаки: id, name, host_id, host_name, last_review</li>
<li>визуализируйте базовые статистики данных: распределения признаков, матрицу попарных корреляций, постройте pair plots</li>
<li>про результатам анализа произведите предобработку переменных</li></ul>

In [1]:
#!pip install numpy pandas seaborn matplotlib plotly scikit-learn kaggle

In [19]:
%%time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')

import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML, display


Wall time: 1e+03 µs


In [38]:
def short_describe(df, sample_size=5, repiter=125):
    repiter='>'*repiter
    nRow, nCol = df.shape
    display(HTML(f'<b>There are {nRow} rows and {nCol} columns</b>'))
    display(HTML("<hr>"))
    d_test=df.sample(n=sample_size, random_state=42).T
    d_test['datatypes']=df.dtypes
    d_describe=df.describe(include='all').T
    
    for i in ['count','min','mean','max','std', 'unique', 'freq']:
        d_test[i]=d_describe[i]
        if i in ['min','mean',]:
            d_test[i].fillna('-', inplace=True)

    d_test['max'].fillna(d_test['unique'], inplace=True)
    d_test['std'].fillna(d_test['freq'], inplace=True)
    d_test.rename(columns = {'max':'max / unique','std':'std / freq'}, inplace = True)
    d_test=d_test.drop(columns=['unique','freq'])
    
    display(d_test)

    
    lst_num = df.select_dtypes(include='object').columns.tolist()
    lst_str = df.select_dtypes(include='number').columns.tolist()
    display(HTML(f'<hr><b>Numeric columns ({len(lst_num)}):</b>'))
    print(lst_num)
    display(HTML(f'<hr><b>String columns ({len(lst_str)}):</b>'))
    print(lst_str)
    display(HTML("<hr>"))
    
    return lst_num, lst_str

In [39]:
%%time

topNrows = 1000
strFileName='AB_NYC_2019.csv'
df = pd.read_csv(strFileName, delimiter=',', nrows = topNrows)

df=df.drop(columns=['id', 'name', 'host_id', 'host_name', 'last_review'])

lst_num, lst_str= short_describe(df)

,521,737,740,660,411,datatypes,count,min,mean,max / unique,std / freq
neighbourhood_group,Brooklyn,Bronx,Manhattan,Manhattan,Brooklyn,object,1000,-,-,5.00000,470.000000
neighbourhood,East New York,Wakefield,Harlem,Harlem,Park Slope,object,1000,-,-,97.00000,112.000000
latitude,40.67497,40.89557,40.79951,40.81068,40.67632,float64,1000.0,40.58147,40.728265,40.89747,0.051538
longitude,-73.87305,-73.8447,-73.95257,-73.94288,-73.97616,float64,1000.0,-74.16534,-73.963382,-73.76133,0.033430
room_type,Entire home/apt,Private room,Entire home/apt,Private room,Entire home/apt,object,1000,-,-,3.00000,580.000000
price,169,50,130,125,250,int64,1000.0,20.0,158.282,3000.00000,152.031708
minimum_nights,3,2,2,1,7,int64,1000.0,1.0,9.137,365.00000,24.284521
number_of_reviews,177,15,35,11,18,int64,1000.0,0.0,79.865,480.00000,88.511169
reviews_per_month,1.86,0.17,0.42,0.12,0.21,float64,959.0,0.01,0.931241,7.14000,0.968909
calculated_host_listings_count,1,1,1,2,1,int64,1000.0,1.0,2.375,28.00000,4.403381


['neighbourhood_group', 'neighbourhood', 'room_type']


['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


Wall time: 32.5 ms


In [ ]:
sns.pairplot(data=tips_raw,vars=["tip","size","total_bill"], hue="sex")

<hr>

### Часть 2. Preprocessing & Feature Engineering

<br>Ваша цель получить как можно более высокие метрики качества (можно взять несколько, R2, MAE, RMSE), сконцентрировавшись на преобразовании признаков.
<br>Опробуйте различные техники:
<ul><li>работа с категориальными переменными (можно начать с dummy);</li>
<li>замена аномалий;</li>
<li>различные варианты шкалирования непрерывных переменных (StandardScaler, RobustScaler, и.т.д.);</li>
<li>обратите внимание на распределение целевой переменной, возможно, с ней тоже можно поработать;</li>
</ul>
Попробуйте на основании имеющихся переменных создать новые, которые могли бы улучшить качество модели. Например, можно найти координаты Манхэттена (самого дорогого района) и при помощи широты и долготы, а также евклидового расстояния создать новую переменную - расстояние от квартиры до этого района. Возможно, такой признак будет работать лучше, чем просто широта и долгота.

<hr>

### Часть 3. Моделирование

<br>Отложите 30% данных для тестирования.
<br>Постройте модели простой линейной регрессии, RidgeCV, LassoCV и ElasticNetCV.
<br>Измерьте качество каждой и визуализируйте важность признаков.
<br>Сделайте интересные выводы :)